# Milestone 1, 2 & 3

In [48]:
import re
import math

### Tokenizer

In [49]:
class Tokenizer:
    def __init__(self):
        self.patterns = [
            (r'\d+\.\d+', lambda x: float(x)),  # Floating-point numbers
            (r'\d+', lambda x: int(x)),          # Integers
            (r'\+', lambda x: x),
            (r'-', lambda x: x),
            (r'\*', lambda x: x),
            (r'/', lambda x: x),
            (r'\(', lambda x: x),
            (r'\)', lambda x: x),
            (r'sqrt', lambda x: x),
            (r'sin', lambda x: x),
            (r'cos', lambda x: x),
            (r'tan', lambda x: x),
            (r'pi', lambda x: math.pi),
        ]

    def tokenize(self, expression):
        tokens = []
        while expression:
            for pattern, handler in self.patterns:
                match = re.match(pattern, expression)
                if match:
                    token = match.group(0)
                    tokens.append(handler(token))
                    expression = expression[len(token):].strip()
                    break
            else:
                raise ValueError("Invalid character in input")
        return tokens

### Parser

In [50]:
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_token_index = 0

    def parse(self):
        try:
            result = self.expression()
            if self.current_token_index != len(self.tokens):
                raise SyntaxError("Invalid syntax")
            return result
        except ZeroDivisionError:
            raise ZeroDivisionError("Division by zero")

    def match(self, expected_token):
        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index] == expected_token:
            self.current_token_index += 1
            return True
        else:
            return False

    def expression(self):
        return self.addition()

    def addition(self):
        result = self.multiplication()

        while self.match('+') or self.match('-'):
            operator = self.tokens[self.current_token_index - 1]
            right_operand = self.multiplication()
            if operator == '+':
                result += right_operand
            else:
                result -= right_operand

        return result

    def multiplication(self):
        result = self.primary()

        while self.match('*') or self.match('/'):
            operator = self.tokens[self.current_token_index - 1]
            right_operand = self.primary()
            if operator == '*':
                result *= right_operand
            else:
                if right_operand == 0:
                    raise ZeroDivisionError("Division by zero")
                result /= right_operand

        return result

    def primary(self):
        if self.match('('):
            result = self.expression()
            if not self.match(')'):
                raise SyntaxError("Invalid syntax")
            return result
        elif isinstance(self.tokens[self.current_token_index], int) or isinstance(self.tokens[self.current_token_index], float):
            result = self.tokens[self.current_token_index]
            self.current_token_index += 1  # Move to the next token
            return result
        elif self.match('sqrt'):
            if not self.match('('):
                raise SyntaxError("Invalid syntax")
            result = self.expression()
            if result < 0:
                raise ValueError("Square root of negative number")
            if not self.match(')'):
                raise SyntaxError("Invalid syntax")
            return math.sqrt(result)
        elif self.match('sin'):
            if not self.match('('):
                raise SyntaxError("Invalid syntax")
            result = self.expression()
            if not self.match(')'):
                raise SyntaxError("Invalid syntax")
            return math.sin(result)
        elif self.match('cos'):
            if not self.match('('):
                raise SyntaxError("Invalid syntax")
            result = self.expression()
            if not self.match(')'):
                raise SyntaxError("Invalid syntax")
            return math.cos(result)
        elif self.match('tan'):
            if not self.match('('):
                raise SyntaxError("Invalid syntax")
            result = self.expression()
            if not self.match(')'):
                raise SyntaxError("Invalid syntax")
            return math.tan(result)
        else:
            raise SyntaxError("Invalid syntax")

# Test the Parser
def test_parser():
    expression = input("Enter an arithmetic expression: ")
    tokenizer = Tokenizer()
    tokens = tokenizer.tokenize(expression)
    if tokens is not None:
        print("Tokens:", tokens)  # Print tokens for debugging
        parser = Parser(tokens)
        try:
            result = parser.parse()
            print("Result:", result)
        except (SyntaxError, ZeroDivisionError, ValueError) as e:
            print("Error:", e)

test_parser()

Tokens: [10, '/', '(', 5, '*', 2, ')']
Result: 1.0
